In [1]:
# Set up for running selenium in Google Colab
## You don't need to run this code if you do it in Jupyter notebook, or other local Python setting
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 2s (144 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
41 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependenc

In [2]:
!pip install chromedriver-autoinstaller

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#chrome_options.add_argument("--disable-javascript")
# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

# set the target URL
url = "put-url-here-to-scrape"

# set up the webdriver
driver = webdriver.Chrome(options=chrome_options)

In [3]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [4]:
url_first = 'https://www.amazon.it/deals?ref_=nav_cs_gb&deals-widget=%257B%2522version%2522%253A1%252C%2522viewIndex%2522%253A'
#page is a mutiple of 6 starting from 0, so page 1 is '' page 2 is '6' page 3 is '12' and so on
page = '6' #page1
url_last = '0%252C%2522presetId%2522%253A%2522deals-collection-lightning-deals%2522%252C%2522dealState%2522%253A%2522UPCOMING%2522%252C%2522dealType%2522%253A%2522LIGHTNING_DEAL%2522%252C%2522sorting%2522%253A%2522FEATURED%2522%257D'
url = url_first + page + url_last
driver.get(url)
html = driver.page_source

In [ ]:
html

In [ ]:
len(html)

869695

In [ ]:
arr = []

In [5]:
from bs4 import BeautifulSoup
from datetime import datetime
import re
import json

soup = BeautifulSoup(html, 'html.parser')
# Find the <div> with the specified ID
div_element = soup.find('div', id='slot-14')

# Find the <script> tag within the <div>
script_content = div_element.find('script').string

# Define the regex pattern
pattern = r'\{"widgetId".*\}\)\n '

# Search for the pattern in the script content
match = re.search(pattern, script_content)

# fore some reason i'm not obtaining the first deal of the day
deals_data = match.group().rstrip(")\n ")
deals = json.loads(deals_data)['prefetchedData']['aapiGetDealsList'][0]['entities']
for deal in deals:
  product_id=deal['entity']['details']['entity']['landingPage']['url'].split("/")[-1]
  start_time= deal['entity']['details']['entity']['startTime']['value']
  #start_time dosen't count early offers for prime members
  time_dic = deal['entity']['badge']['entity']['messaging']['content']['fragments'][1]
  actual_start_time= time_dic['dateTime'] if 'dateTime' in time_dic else time_dic['countdownTimer']['targetTime']
  print(product_id + " : " + start_time + " : " + actual_start_time)

B0B5BZFLYW : 2024-03-11T07:35:00.000Z : 2024-03-11T07:05:00.000Z
B07XXTY79M : 2024-03-11T07:35:00.000Z : 2024-03-11T07:05:00.000Z
B09PYKV61M : 2024-03-11T07:40:00.000Z : 2024-03-11T07:10:00.000Z
B09JKGH463 : 2024-03-11T07:40:00.000Z : 2024-03-11T07:10:00.000Z
B0B6F4L4YX : 2024-03-11T07:40:00.000Z : 2024-03-11T07:10:00.000Z
B09QJ9Q35V : 2024-03-11T07:40:00.000Z : 2024-03-11T07:10:00.000Z
B0BR3939T2 : 2024-03-11T07:45:00.000Z : 2024-03-11T07:15:00.000Z
B0CDC6NFPV : 2024-03-11T07:45:00.000Z : 2024-03-11T07:15:00.000Z
B08613LJDQ : 2024-03-11T07:45:00.000Z : 2024-03-11T07:15:00.000Z
B0B5V2VS6C : 2024-03-11T07:45:00.000Z : 2024-03-11T07:15:00.000Z
B08SW2FC58 : 2024-03-11T07:50:00.000Z : 2024-03-11T07:20:00.000Z
B085ZY4S5X : 2024-03-11T07:55:00.000Z : 2024-03-11T07:25:00.000Z
B0CKXY8TGH : 2024-03-11T07:55:00.000Z : 2024-03-11T07:25:00.000Z
B08N6BND31 : 2024-03-11T08:00:00.000Z : 2024-03-11T07:30:00.000Z
B0BWCFZXW1 : 2024-03-11T08:00:00.000Z : 2024-03-11T07:30:00.000Z
B09HZN2Y4M : 2024-03-11T0

In [ ]:
time_dic

{'dateTime': '2024-03-11T06:30:00.000Z'}

In [27]:
test_product_url = "https://www.amazon.it/dp/B0CQCLF3HW"
driver.get(test_product_url)
driver.page_source

'<html><head>\n    <meta http-equiv="content-type" content="text/html; charset=UTF-8">\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<title>Ci dispiace</title>\n</head>\n<body bgcolor="#FFFFFF" text="#000000">\n\n<!--\nPer discutere l\'accesso automatizzato ai dati di Amazon, contatta api-services-support@amazon.com.\nPer informazioni sulla migrazione alle nostre API, fare riferimento alle API di Marketplace all\'indirizzo https://developer.amazonservices.com/ref=rm_5_sv o alla nostra Product Advertising API all\'indirizzo https://affiliate-program.amazon.com/gp/advertising/api/detail/main.html/ref=rm_5_ac per i casi d\'uso pubblicitari.\n\nTo discuss automated access to Amazon data please contact api-services-support@amazon.com.\nFor information about migrating to our APIs refer to our Marketplace APIs at https://developer.amazonservices.it/ref=rm_c_sv, or our Product Advertising API at https://programma-affiliazione.amazon.it/gp/adve

In [ ]:
#questo metodo funzionva, ma a caso da selenium non si trova più l'url "/promo/", dal sorgente chrome c'è ancora
test_product_url = "https://www.amazon.it/dp/B0CQCLF3HW"
driver.get(test_product_url)
html = driver.page_source
pattern = r'/promo/[^"]*'
promos = re.findall(pattern, html)
[promo.split("/")[-1] for promo in promos]

[]

In [ ]:
html

'<html><head>\n    <meta http-equiv="content-type" content="text/html; charset=UTF-8">\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<title>Ci dispiace</title>\n</head>\n<body bgcolor="#FFFFFF" text="#000000">\n\n<!--\nPer discutere l\'accesso automatizzato ai dati di Amazon, contatta api-services-support@amazon.com.\nPer informazioni sulla migrazione alle nostre API, fare riferimento alle API di Marketplace all\'indirizzo https://developer.amazonservices.com/ref=rm_5_sv o alla nostra Product Advertising API all\'indirizzo https://affiliate-program.amazon.com/gp/advertising/api/detail/main.html/ref=rm_5_ac per i casi d\'uso pubblicitari.\n\nTo discuss automated access to Amazon data please contact api-services-support@amazon.com.\nFor information about migrating to our APIs refer to our Marketplace APIs at https://developer.amazonservices.it/ref=rm_c_sv, or our Product Advertising API at https://programma-affiliazione.amazon.it/gp/adve

In [ ]:
driver.implicitly_wait(5)
driver.get("https://www.amazon.it/dp/B0C2V1113J")

html = driver.page_source
pattern = r'/promo/[^"]*'
promos = re.findall(pattern, html)
[promo.split("/")[-1] for promo in promos]

[]

In [ ]:
import requests
def get_promo_details(promo_id):
  base_promo_url = 'https://www.amazon.it/promotion/psp/'

  promo_url = base_promo_url + promo_id
  #promo_data = requests.get(promo_url).json()['cartProgressIndicatorResponse']['promotionTiers'][0]
  html = requests.get(promo_url).text
  pattern = r'let getCartItemsRequest = ({.*?});'
  promo_data = re.search(pattern, html, re.DOTALL).group(1)
  print(promo_data)
  #return {"promo_type": promo_data['benefitType'].lower(), "value": promo_data['benefitValue']}

get_promo_details('A3O2Q4VBLX7WDT')


{
                    "promotionId": "A3O2Q4VBLX7WDT",
                    "thresholdType": "QUANTITY",
                    "minQuantity": "1",
                    "minAmount": "",
                    "benefitAmount": "",
                    "benefitPercent": "5",
                    "benefitAmountCap": "",
                    "showCartProgressIndicator": false,
                    "numberOfProductSelections" : "2",
                    "promotionThresholds": JSON.stringify(promotionThresholdsArray)
                }


In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime

soup = BeautifulSoup(html, 'html.parser')

# Find the parent div with the specified class prefix
parent_div = soup.select_one('div[class^="Grid-module__gridDisplayGrid"]')

# Find all child divs with the specified class prefix
divs = parent_div.select('div[class^="DealGridItem-module__dealItemDisplayGrid"]')

for div in divs:
    # Extract product URL
    product_url = div.select_one('a[class^="a-link-normal DealCard-module__linkOutlineOffset"]')['href']

    # Extract offer time and convert to datetime
    offer_time_str = div.select_one('time[class^="DealMessaging-module__dealMessaging"]').text.split(' ')[-1]
    offer_time = None
    try:
      #offer_time = datetime.strptime(offer_time_str, '%H:%M')
      offer_time = offer_time_str
    except:
      next

    print(f'Product URL: {product_url}')
    print(f'Offer Time: {offer_time}')


Product URL: https://www.amazon.it/telescopica-foratura-armadio-progetti-diametro/dp/B0B2P6XFCH?pf_rd_r=ZBDJ72Q8EEP923K4QK63&pf_rd_t=Events&pf_rd_i=deals&pf_rd_p=625f4ee2-9c7d-4cb3-8da6-d5915b990e77&pf_rd_s=slot-14&ref=dlx_deals_gd_dcl_img_60_36750c5c_dt_sl14_77
Offer Time: tempo
Product URL: https://www.amazon.it/YD-LIFE-Illuminazione-Movimento-Impermeabile/dp/B0CMV2CTNQ?pf_rd_r=ZBDJ72Q8EEP923K4QK63&pf_rd_t=Events&pf_rd_i=deals&pf_rd_p=625f4ee2-9c7d-4cb3-8da6-d5915b990e77&pf_rd_s=slot-14&ref=dlx_deals_gd_dcl_img_61_033fd6ba_dt_sl14_77
Offer Time: tempo
Product URL: https://www.amazon.it/OBVHNUA-Torretta-Multipresa-Verticale-Elettrica/dp/B09SPQPV44?pf_rd_r=ZBDJ72Q8EEP923K4QK63&pf_rd_t=Events&pf_rd_i=deals&pf_rd_p=625f4ee2-9c7d-4cb3-8da6-d5915b990e77&pf_rd_s=slot-14&ref=dlx_deals_gd_dcl_img_62_fa4b78dd_dt_sl14_77
Offer Time: tempo
Product URL: https://www.amazon.it/Tigratigro-Pennini-sensibilit%C3%A0-capacitiva-Universale/dp/B0B9N7QY1R?pf_rd_r=ZBDJ72Q8EEP923K4QK63&pf_rd_t=Events&pf_rd_i